<a href="https://colab.research.google.com/github/yeison34k/diplomado_analisis_datos_machine_learning_python/blob/main/Notebook_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

# **Diplomado de Análisis de datos y Machine Learning en Python**


El presente diplomado hace parte del centro de Big Data de la facultad de ciencias exactas y naturales (FCEN) de la Universidad de Antioquia.

## **Sesión 10**

## **Contenido**

- <a href="#pan"> Operaciones sobre los datos en Pandas</a><br>
- <a href="#ope"> Operaciones entre DataFrames y Series</a><br>
- <a href="#agg"> Agregación y agrupamiento</a><br>
  - <a href="#gro"> Groupby</a><br>

<p><a name="pan"></a></p>

# **Operaciones sobre los datos en Pandas**

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones vectorizadas, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.). Pandas hereda gran parte de esta funcionalidad de NumPy.

Sin embargo, Pandas incluye un par de elementos útiles: para operaciones unarias como funciones de negación y trigonométricas, estas ufuncs conservarán etiquetas de índice y columna en la salida, y para operaciones binarias como suma y multiplicación, Pandas alineará automáticamente los índices. Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en objetos Pandas Series y DataFrame. 

Comencemos por definir una serie y un DataFrame simples con los cuales demostrar esto:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(42)
s = pd.Series(np.random.randint(0,25,4))
df = pd.DataFrame(np.random.randint(0,25,(4,4)))
s


0     6
1    19
2    14
3    10
dtype: int64

In [ ]:
df

,0,1,2,3
0,7,20,6,18
1,22,10,10,23
2,20,3,7,23
3,2,21,20,1


**Preservando el índice**

Si aplicamos un ufunc de NumPy en cualquiera de estos objetos, el resultado será otro objeto de Pandas con los índices preservados:


In [ ]:
np.exp(s)

0    4.034288e+02
1    1.784823e+08
2    1.202604e+06
3    2.202647e+04
dtype: float64

In [ ]:
np.log(df)

,0,1,2,3
0,1.945910,2.995732,1.791759,2.890372
1,3.091042,2.302585,2.302585,3.135494
2,2.995732,1.098612,1.945910,3.135494
3,0.693147,3.044522,2.995732,0.000000


**Alineación del índice**


**Series** 

Para operaciones binarias con dos objetos (Series o DataFrames), Pandas alineará los índices en el proceso de realizar la operación.


In [ ]:
d_poblacion = {'Brasil': 210147125, 'Colombia': 50372424, 'Argentina' : 44938712}
d_area = {'Brasil': 8514877, 'Colombia': 1141748, 'Argentina' : 2792600}

s_poblacion = pd.Series(d_poblacion)
s_area = pd.Series(d_area)
s_poblacion

Brasil       210147125
Colombia      50372424
Argentina     44938712
dtype: int64

In [ ]:
s_area

Brasil       8514877
Colombia     1141748
Argentina    2792600
dtype: int64

Veamos qué sucede cuando dividimos estas series para calcular la densidad de población:

In [ ]:
s_poblacion / s_area

Brasil       24.679995
Colombia     44.118688
Argentina    16.092069
dtype: float64

In [ ]:
d_poblacion = {'Brasil': 210147125, 'Colombia': 50372424, 'Argentina' : 44938712}
d_area = {'Brasil': 8514877, 'Colombia': 1141748, 'Peru' : 2792600}

s_poblacion = pd.Series(d_poblacion)
s_area = pd.Series(d_area)
s_poblacion

Brasil       210147125
Colombia      50372424
Argentina     44938712
dtype: int64

In [ ]:
s_area

Brasil      8514877
Colombia    1141748
Peru        2792600
dtype: int64

In [ ]:
s_poblacion / s_area

Argentina          NaN
Brasil       24.679995
Colombia     44.118688
Peru               NaN
dtype: float64

El arreglo resultante contiene la unión de los índices de los dos arreglos de entrada, que podrían determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:


In [ ]:
s_poblacion.index | s_area.index

Index(['Argentina', 'Brasil', 'Colombia', 'Peru'], dtype='object')

Cualquier elemento para el que una serie u otra no tenga una entrada se marca con `NaN` ("Not a Number"), que es la forma en que Pandas marca los datos faltantes.

Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores faltantes se rellenan con `NaN` de forma predeterminada

In [ ]:
A = pd.Series([2, 4, 6])
B = pd.Series([1, 3, 5], index = [1, 2, 3])
A

0    2
1    4
2    6
dtype: int64

In [ ]:
B

1    1
2    3
3    5
dtype: int64

In [ ]:
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si no queremos incluir `NaN` en la salida, podemos utilizar la ufunc directamente. Por ejemplo, llamar a `A.add(B)` es equivalente a llamar a `A + B`, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en `A` o `B` que pueda faltar, mediante al kwarg `fill_value`:

In [ ]:
A.add(B, fill_value= 0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

In [ ]:
A

0    2
1    4
2    6
dtype: int64

In [ ]:
B

1    1
2    3
3    5
dtype: int64

Note que este kwarg reemplaza `0` en los valores faltantes en las series, no directamente los `NaN` en la salida de la operación de suma.

**DataFrames**

Cuando se realizan operaciones en DataFrames se produce un tipo similar de alineación tanto para las columnas como para los índices:


In [ ]:
np.random.seed(1)
df1 = df.copy()
df1.columns = [f'col{i}' for i in range(4)]
df2 = pd.DataFrame(np.random.randint(0,25,(4,4)), columns = [f'col{i}' for i in range(1,5)])
df2

,col1,col2,col3,col4
0,5,11,12,8
1,9,11,5,15
2,0,16,1,12
3,7,13,6,18


In [ ]:
df1

,col0,col1,col2,col3
0,7,20,6,18
1,22,10,10,23
2,20,3,7,23
3,2,21,20,1


In [ ]:
df1 + df2

,col0,col1,col2,col3,col4
0,NaN,25,17,30,NaN
1,NaN,19,21,28,NaN
2,NaN,3,23,24,NaN
3,NaN,28,33,7,NaN


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos, y los índices del resultado están ordenados. Como con las series, podemos usar la ufunc asociada y utilizar el kwarg`fill_value` 


In [ ]:
df1.add(df2, fill_value= 1000)

,col0,col1,col2,col3,col4
0,1007.0,25,17,30,1008.0
1,1022.0,19,21,28,1015.0
2,1020.0,3,23,24,1012.0
3,1002.0,28,33,7,1018.0


<p><a name="ope"></a></p>

# **Operaciones entre DataFrame y Series**

Al realizar operaciones entre un DataFrame y una Serie, la alineación del índice y la columna se mantiene de manera similar. Las operaciones entre un DataFrame y una Serie son similares a las operaciones entre un arreglo de NumPy bidimensional y uno unidimensional.

La resta entre un un arreglo bidimensional y una de sus filas se aplica por filas. En Pandas, la convención opera de manera similar:

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.arange(9).reshape(3,3), columns = ['a','b','c'])
s = pd.Series(np.arange(3), index = ['a','b','c'])
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [ ]:
s

a    0
b    1
c    2
dtype: int64

In [ ]:
df - s

,a,b,c
0,0,0,0
1,3,3,3
2,6,6,6


Si, en cambio, deseamos operar sobre las columnas, podemos especificar el eje explícitamente con el kwarg `axis`. Note que en este caso los índices de la fila deben coincidir con los índices del DataFrame, a diferencia del caso anterior en el que los índices de la Series coincidia con las columnas del DataFrame:


In [ ]:
df.subtract(s, axis = 1)

,a,b,c
0,0,0,0
1,3,3,3
2,6,6,6


In [ ]:
df.subtract(s, axis = 0)

,a,b,c
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
a,NaN,NaN,NaN
b,NaN,NaN,NaN
c,NaN,NaN,NaN


In [ ]:
s.reset_index(drop= True,inplace=True)
df.subtract(s, axis = 0)

,a,b,c
0,0,1,2
1,2,3,4
2,4,5,6


Estudiemos este tipo de operaciones utilizando un conjunto de datos real. En este caso utilizaremos el conjunto de datos `auto.csv` que contiene muestras de vehículos con una serie de características:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tomasate/Diplomado_ML/main/datasets/data/nivel_1/Data/auto.csv')
df

,Unnamed: 0,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0,10.217391,Medium,0,1
197,197,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.955556,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0,12.368421,High,0,1
198,198,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0,13.055556,Medium,0,1
199,199,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0,9.038462,Medium,1,0


Para comenzar, note que la columna `Unnamed: 0` corresponde a la información del índice, por lo que es una columna que no debemos incluir en el conjunto de datos.

Con el método `drop` podemos eliminar una columna, una fila, o una combinación de ambas (mirar documentación del método). El método no es *in place* por lo que debemos utilizar el kwarg `inplace` para hacer el cambio efectivo sobre el DataFrame

In [ ]:
df.drop(columns='Unnamed: 0', inplace= True)
df

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0,10.217391,Medium,0,1
197,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.955556,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0,12.368421,High,0,1
198,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0,13.055556,Medium,0,1
199,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0,9.038462,Medium,1,0


Otra opción es utilizar el kwarg `index_col` de la función `read_csv` con el cual definimos cuál será el índice del DataFrame:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tomasate/Diplomado_ML/main/datasets/data/nivel_1/Data/auto.csv', index_col='Unnamed: 0')
df

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0,10.217391,Medium,0,1
197,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.955556,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0,12.368421,High,0,1
198,-1,95,volvo,std,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0,13.055556,Medium,0,1
199,-1,95,volvo,turbo,four,sedan,rwd,front,109.1,0.907256,0.956944,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0,9.038462,Medium,1,0


Una vez eliminada esta columna, realicemos algunas operaciones útiles.

Por ejemplo, si quisieramos construir una nueva columna que nos muestre la relación *horsepower* / *price* podemos utilizar simplemente una operación vectorizada utilizando el operador asociado a la ufunc:

In [ ]:
df['ratio'] = df['horsepower'] / df['price']
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio
0,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,0.008225
1,3,122,alfa-romero,std,two,convertible,rwd,front,88.6,0.811148,0.890278,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,0.006727
2,1,122,alfa-romero,std,two,hatchback,rwd,front,94.5,0.822681,0.909722,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,0.009333
3,2,164,audi,std,four,sedan,fwd,front,99.8,0.848630,0.919444,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,0.007312
4,2,164,audi,std,four,sedan,4wd,front,99.4,0.848630,0.922222,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,0.006590


In [ ]:
df.shape

(201, 30)

In [ ]:
df['horsepower-binned'].unique()

array(['Medium', 'Low', 'High', nan], dtype=object)

Para ordenar esta columna podemos utilizar el método `sort_values`. Con el kwarg `ascending` podemos controlar si el ordenamiento es ascendente o no:

In [ ]:
df.ratio.sort_values(ascending=False)

163    0.013729
134    0.013482
76     0.013266
47     0.013090
22     0.012819
         ...   
66     0.004365
65     0.004354
71     0.004053
63     0.003925
67     0.003892
Name: ratio, Length: 201, dtype: float64

Vemos que la observación con índice 163 contiene la mejor relación *horsepower* / *price*. Accedamos a esta observación, mirando por ejemplo la columna asociada a la constructora *make*:

In [ ]:
df.loc[163,'make']

'toyota'

In [ ]:
df.iloc[163,2]

'toyota'

En muchas ocaciones vamos a querer acceder a esta información, sin tener la necesidad de realizar una visualización para obtener el índice asociado a la observación con el valor mayor. Para este fin podemos utilizar el método `idxmax`:

In [ ]:
id_max = df.ratio.idxmax()
df.loc[id_max, 'make']

'toyota'

Otra operación común que se aplica sobre los datos numéricos es el da la normalización de una variable (observe que el conjunto de datos ya contiene este tipo de datos numéricos). Podemos visualizar rápidamente una variable utilizando una operación vectorizada similar a la anterior. Hagamos esto para la variable `height`:

In [ ]:
df.height.describe()

count    201.000000
mean      53.766667
std        2.447822
min       47.800000
25%       52.000000
50%       54.100000
75%       55.500000
max       59.800000
Name: height, dtype: float64

In [ ]:
df.height = df.height / df.height.max()

In [ ]:
df.height.describe()

count    201.000000
mean       0.899108
std        0.040933
min        0.799331
25%        0.869565
50%        0.904682
75%        0.928094
max        1.000000
Name: height, dtype: float64

Otro tipo de operación común es la de asignar un valor numérico a una variable categórica de tipo `str`. Por ejemplo, si quisieramos modificar la variable `num-of-doors` 



In [ ]:
df['num-of-doors'].unique()

array(['two', 'four'], dtype=object)

Pasando de tener los valores "four" y "two" a 4 y 2, respectivamente, podemos utilizar la función nativa de Python `map(f, a)`. Esta toma una lista o arreglo `a` y le aplica alguna función `f` a cada uno de los elementos de `a` (similar a lo que hacen las ufuncs de NumPy). 

Veamos un ejemplo: creemos el arreglo `[1, 2, 3, 4]` y apliquemos `map` pasando como argumento una función que devuelva el cuadrado de los elementos: 

In [ ]:
lista = [1, 2, 3, 4]
def f(x):
  return x ** 2

list(map(f, lista))

[1, 4, 9, 16]

Note que `map` no devuelve una lista sino un generador, por lo que utilizamos el constructor `list` para obtener una lista. 

También podemos utilizar esta función para definir un mapeo de una variable a otra, mediante el uso de un diccionario, donde las claves representarán el valor que contiene el arreglo, y las claves los nuevos valores que les queremos dar. 

En el caso de una serie, podemos utilizar `map` como un método del objeto



In [ ]:
df['num-of-doors'].map({'two':2, 'four':4})

0      2
1      2
2      2
3      4
4      4
      ..
196    4
197    4
198    4
199    4
200    4
Name: num-of-doors, Length: 201, dtype: int64

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
df['num-of-doors'] = df['num-of-doors'].map({'two':2, 'four':4})
df.sample(6)

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio
95,2,168,nissan,std,2,hardtop,fwd,front,95.1,0.780394,0.886111,0.891304,2008,ohc,four,97,2bbl,3.15,3.29,9.40,69.0,5200.0,31,37,8249.0,7.580645,Low,0,1,0.008365
15,0,122,bmw,std,2,sedan,rwd,front,103.5,0.931283,0.943056,0.897993,3380,ohc,six,209,mpfi,3.62,3.39,8.00,182.0,5400.0,16,22,41315.0,14.687500,High,0,1,0.004405
30,2,137,honda,std,2,hatchback,fwd,front,86.6,0.694858,0.887500,0.849498,1819,ohc,four,92,1bbl,2.91,3.41,9.20,76.0,6000.0,31,38,6855.0,7.580645,Low,0,1,0.011087
158,0,91,toyota,std,4,sedan,fwd,front,95.7,0.799135,0.894444,0.882943,2140,ohc,four,98,2bbl,3.19,3.03,9.00,70.0,4800.0,28,34,9258.0,8.392857,Low,0,1,0.007561
128,3,150,saab,std,2,hatchback,fwd,front,99.1,0.896684,0.923611,0.938127,2658,ohc,four,121,mpfi,3.54,3.07,9.31,110.0,5250.0,21,28,11850.0,11.190476,Medium,0,1,0.009283
115,1,119,plymouth,std,2,hatchback,fwd,front,93.7,0.755887,0.886111,0.849498,1918,ohc,four,90,2bbl,2.97,3.23,9.40,68.0,5500.0,37,41,5572.0,6.351351,Low,0,1,0.012204


Más adelante veremos formas alternativas para realizar este tipo de operaciones de manera que las podamos incluir en un proceso más general.

<p><a name="agg"></a></p>

# **Agregaciones y agrupamiento**

Una parte esencial del análisis de grandes cantidades de datos es el cálculo de agregaciones como la suma, media, mediana, etc; que nos proporcionan un resumen estadístico de los datos. En esta sección, exploraremos agregaciones en Pandas.

En la sesión anterior vimos cómo aplicar funciones de agregación simples sobre los objetos de pandas, similares a las que hemos visto en los arreglos de NumPy. Veamos ahora algunas funcionalidades más complejas:



**Aggregate (agg)**: 

Este método permite tener mucha más flexibilidad en las operaciones de agregación. Puede tomar una cadena de caracteres, una función o una lista de las mismas y calcular todos las agregaciones a la vez. Veamos algunos ejemplos:

In [ ]:
df.price.agg([np.mean, np.max, np.min])

mean    13207.129353
amax    45400.000000
amin     5118.000000
Name: price, dtype: float64

In [ ]:
df.price.agg(["mean", 'min', "count"])

mean     13207.129353
min       5118.000000
count      201.000000
Name: price, dtype: float64

In [ ]:
df.make.agg(['count', 'nunique'])

count      201
nunique     22
Name: make, dtype: int64

También lo podemos aplicar sobre el dataframe completo y utilizar un diccionario para mapear qué funciones (valores) queremos aplicar a una columna particular (claves)

In [ ]:
df.agg({'price': ["mean", "count"], 'make':'count'})

,price,make
count,201.000000,201.0
mean,13207.129353,NaN


**Apply**

Este método nos permite aplicar una función arbitraria. La función debe tomar un DataFrame y devolver un objeto de Pandas o un número.

Por ejemplo, supongamos que queremos pasar de tener la variable continua *price* a una variable categórica que tome los valores *inf* y *sup* de acuerdo a un criterio particular. Podemos utilizar la media como criterio de separación:

In [ ]:
df.price.describe()

count      201.000000
mean     13207.129353
std       7947.066342
min       5118.000000
25%       7775.000000
50%      10295.000000
75%      16500.000000
max      45400.000000
Name: price, dtype: float64

In [ ]:
media = df.price.mean()

In [ ]:
def categoria(df,colu, mean = media):
  if df[colu] >= mean:
    return "sup"
  else:
    return "inf"

Pasemos esta función al método indicando que queremos realizar la operación por filas:

In [ ]:
df.apply(categoria, axis= 1, args=['price', media]).unique()

array(['sup', 'inf'], dtype=object)

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
df['cat_price'] = df.apply(categoria, axis= 1, args=['price', media])
df.sample(6)

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio,cat_price
177,-1,122,toyota,std,4,wagon,rwd,front,104.5,0.902451,0.923611,0.904682,3151,dohc,six,161,mpfi,3.27,3.35,9.2,156.0,5200.0,19,24,15750.0,12.368421,High,0,1,0.009905,sup
33,1,101,honda,std,2,hatchback,fwd,front,93.7,0.720807,0.888889,0.879599,1956,ohc,four,92,1bbl,2.91,3.41,9.2,76.0,6000.0,30,34,7129.0,7.833333,Low,0,1,0.010661,inf
121,3,122,plymouth,turbo,2,hatchback,rwd,front,95.9,0.832292,0.920833,0.839465,2818,ohc,four,156,spdi,3.59,3.86,7.0,145.0,5000.0,19,24,12764.0,12.368421,Medium,0,1,0.011360,inf
181,2,94,volkswagen,std,4,sedan,fwd,front,97.3,0.825084,0.909722,0.931438,2212,ohc,four,109,mpfi,3.19,3.40,9.0,85.0,5250.0,27,34,8195.0,8.703704,Low,0,1,0.010372,inf
102,3,194,nissan,turbo,2,hatchback,rwd,front,91.3,0.820279,0.943056,0.831104,3139,ohcv,six,181,mpfi,3.43,3.27,7.8,200.0,5200.0,17,23,19699.0,13.823529,High,0,1,0.010153,sup
135,2,83,subaru,std,2,hatchback,fwd,front,93.7,0.758770,0.883333,0.897993,2120,ohcf,four,108,2bbl,3.62,2.64,8.7,73.0,4400.0,26,31,7053.0,9.038462,Low,0,1,0.010350,inf


Note que en la función que se construya para pasar al método `apply`, se pueden evaluar condiciones sobre múltiples columnas para realizar una operación, aunque esta sólo actúe sobre una de ellas. Más adelante veremos ejemplos donde esto pueda ser útil.

Este tipo de operación, aunque nos sirvió para ilustrar el método, se puede realizar fácilmente con la función `cut` de Pandas, la cual nos permite segmentar una variable (*price* en este caso) en ciertos *bins*, a los cuales podemos asociar de forma opcional una etiqueta mediante el kwarg `labels`. 

Por ejemplo, realicemos la conversión anterior pero ahora incluyendo un tercer valor adicional `med`:

In [ ]:
pd.cut(df.price, 3, labels= ['inf', 'med', 'sup'])

0      inf
1      inf
2      inf
3      inf
4      inf
      ... 
196    inf
197    med
198    med
199    med
200    med
Name: price, Length: 201, dtype: category
Categories (3, object): ['inf' < 'med' < 'sup']

In [ ]:
pd.cut(df.price, 3, labels= ['inf', 'med', 'sup']).value_counts()

inf    171
med     18
sup     12
Name: price, dtype: int64

Modifiquemos la variable en nuestro conjunto de datos:

In [ ]:
print(df.price.min(), df.price.max())
dif = df.price.max() -df.price.min()
print(dif/3)
print(df.price.min()+dif/3 , df.price.min()+2*dif/3 , df.price.min()+3*dif/3)

5118.0 45400.0
13427.333333333334
18545.333333333336 31972.666666666668 45400.0


In [ ]:
df['cat_price'] = pd.cut(df.price, 3, labels= ['inf', 'med', 'sup'])
df.sample(6)

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio,cat_price
161,1,168,toyota,std,2,sedan,rwd,front,94.5,0.810668,0.888889,0.879599,2265,dohc,four,98,mpfi,3.24,3.08,9.4,112.0,6600.0,26,29,9298.0,9.038462,Medium,0,1,0.012046,inf
14,0,122,bmw,std,4,sedan,rwd,front,103.5,0.908217,0.929167,0.931438,3230,ohc,six,209,mpfi,3.62,3.39,8.0,182.0,5400.0,16,22,30760.0,14.687500,High,0,1,0.005917,med
56,1,129,mazda,std,2,hatchback,fwd,front,98.8,0.854397,0.923611,0.897993,2385,ohc,four,122,2bbl,3.39,3.39,8.6,84.0,4800.0,26,32,8845.0,9.038462,Low,0,1,0.009497,inf
19,0,81,chevrolet,std,4,sedan,fwd,front,94.5,0.763095,0.883333,0.869565,1909,ohc,four,90,2bbl,3.03,3.11,9.6,70.0,5400.0,38,43,6575.0,6.184211,Low,0,1,0.010646,inf
176,-1,90,toyota,std,4,sedan,rwd,front,104.5,0.902451,0.923611,0.904682,3131,dohc,six,171,mpfi,3.27,3.35,9.2,156.0,5200.0,20,24,15690.0,11.750000,High,0,1,0.009943,inf
140,0,102,subaru,std,4,sedan,4wd,front,97.0,0.826526,0.908333,0.908027,2385,ohcf,four,108,2bbl,3.62,2.64,9.0,82.0,4800.0,24,25,9233.0,9.791667,Low,0,1,0.008881,inf


<p><a name="gro"></a></p>

## **GroupBy: dividir, aplicar y combinar**

Las agregaciones simples pueden darnos una idea del conjunto de datos, pero a menudo preferimos agregar condicionalmente sobre alguna etiqueta o índice: esto se implementa en la llamada operación *groupby*

El objeto GroupBy es una abstracción muy flexible. En muchos sentidos, podemos tratarlo como si fuera una colección de DataFrames, para los cuales se realizan una serie de operaciones complejas "por debajo".

En la siguiente figura se ilustra un ejemplo canónico de esta operación dividir-aplicar-combinar, donde "aplicar" hace referencia a la función de agregación *apply*:

![groupby](https://lewtun.github.io/dslectures/images/split-apply-combine.png)

* El paso de división implica dividir y agrupar un DataFrame según el valor de la clave especificada.
* El paso de aplicación implica calcular alguna función, generalmente una agregación, dentro de los grupos individuales.
* El paso de combinación fusiona los resultados de estas operaciones en un objeto (Serie o DataFrame) como salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de las funcionalidades de enmascaramiento, agregación y fusión descritos anteriormente, lo importante es que las divisiones intermedias no necesitan ser instanciadas explícitamente. El poder de GroupBy es que abstrae estos pasos: el usuario no necesita pensar en cómo se realiza el cálculo "por debajo", sino que piensa en la operación como un todo.

Veamos un ejemplo concreto con el conjunto de datos que hemos cargado



In [ ]:
df.head()

,symboling,normalized-losses,make,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio,cat_price
0,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0,11.190476,Medium,0,1,0.008225,inf
1,3,122,alfa-romero,std,2,convertible,rwd,front,88.6,0.811148,0.890278,0.816054,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0,11.190476,Medium,0,1,0.006727,inf
2,1,122,alfa-romero,std,2,hatchback,rwd,front,94.5,0.822681,0.909722,0.876254,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0,12.368421,Medium,0,1,0.009333,inf
3,2,164,audi,std,4,sedan,fwd,front,99.8,0.848630,0.919444,0.908027,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0,9.791667,Medium,0,1,0.007312,inf
4,2,164,audi,std,4,sedan,4wd,front,99.4,0.848630,0.922222,0.908027,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0,13.055556,Medium,0,1,0.006590,inf


Elijamos como clave de agrupación la variable `make`:

In [ ]:
df.groupby('make')

Note que lo que se devuelve no es un conjunto de DataFrames, sino un objeto `DataFrameGroupBy`. Para producir un resultado, podemos aplicar un agregado a este objeto, que realizará los pasos apropiados de aplicación / combinación para producir el resultado deseado. Por ejemplo apliquemos la función de agregación `count`:


In [ ]:
df.groupby('make').count()

,symboling,normalized-losses,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,city-L/100km,horsepower-binned,diesel,gas,ratio,cat_price
make,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alfa-romero,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
audi,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
bmw,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
chevrolet,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
dodge,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
honda,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
isuzu,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
jaguar,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,3,3,3,3
mazda,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,13,17,17,17,17,17,17,17,17,17,17,17,17


Note que obtenemos como salida cuántos elementos, por constructora, aparecen en cada columna. Obviamente todas las columnas tienen el mismo valor, por lo que podemos seleccionar alguna columna que queramos visualizar

In [ ]:
df.groupby('make').symboling.count()

make
alfa-romero       3
audi              6
bmw               8
chevrolet         3
dodge             9
honda            13
isuzu             2
jaguar            3
mazda            17
mercedes-benz     8
mercury           1
mitsubishi       13
nissan           18
peugot           11
plymouth          7
porsche           4
renault           2
saab              6
subaru           12
toyota           32
volkswagen       12
volvo            11
Name: symboling, dtype: int64

In [ ]:
df.groupby('make')[["symboling"]].count()

,symboling
make,
alfa-romero,3
audi,6
bmw,8
chevrolet,3
dodge,9
honda,13
isuzu,2
jaguar,3
mazda,17


Alternativamente podemos utilizar el método `size`:

In [ ]:
df.groupby('make').size()

make
alfa-romero       3
audi              6
bmw               8
chevrolet         3
dodge             9
honda            13
isuzu             2
jaguar            3
mazda            17
mercedes-benz     8
mercury           1
mitsubishi       13
nissan           18
peugot           11
plymouth          7
porsche           4
renault           2
saab              6
subaru           12
toyota           32
volkswagen       12
volvo            11
dtype: int64

Si ordenamos esta salida en forma descendente obtendremos la misma salida de la aplicación del método `value_counts` sobre la serie `df.make`:

In [ ]:
df.groupby('make').size().sort_values(ascending= False)

make
toyota           32
nissan           18
mazda            17
honda            13
mitsubishi       13
subaru           12
volkswagen       12
volvo            11
peugot           11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
audi              6
saab              6
porsche           4
chevrolet         3
alfa-romero       3
jaguar            3
isuzu             2
renault           2
mercury           1
dtype: int64

In [ ]:
df.make.value_counts()

toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
subaru           12
volkswagen       12
peugot           11
volvo            11
dodge             9
mercedes-benz     8
bmw               8
plymouth          7
saab              6
audi              6
porsche           4
alfa-romero       3
jaguar            3
chevrolet         3
renault           2
isuzu             2
mercury           1
Name: make, dtype: int64

En este caso obviamente la mejor opción es utilizar el método `value_counts`, lo importante es ver la libertad que nos proporciona el `groupby` para agrupar y realizar operaciones sobre los datos.

Veamos algunos otros ejemplos:

In [ ]:
df.groupby('make').price.max()

make
alfa-romero      16500.0
audi             23875.0
bmw              41315.0
chevrolet         6575.0
dodge            12964.0
honda            12945.0
isuzu            11048.0
jaguar           36000.0
mazda            18344.0
mercedes-benz    45400.0
mercury          16503.0
mitsubishi       14869.0
nissan           19699.0
peugot           18150.0
plymouth         12764.0
porsche          37028.0
renault           9895.0
saab             18620.0
subaru           11694.0
toyota           17669.0
volkswagen       13845.0
volvo            22625.0
Name: price, dtype: float64

In [ ]:
df.groupby('make').price.agg(["min","max"]).sort_values(ascending= False,by='min')

,min,max
make,,
jaguar,32250.0,36000.0
mercedes-benz,25552.0,45400.0
porsche,22018.0,37028.0
mercury,16503.0,16503.0
bmw,16430.0,41315.0
audi,13950.0,23875.0
alfa-romero,13495.0,16500.0
volvo,12940.0,22625.0
peugot,11900.0,18150.0


Incluyamos ahora otra función que nos mida la diferencia entre el precio máximo y el mínimo, y ordenemos de forma descendente respecto a esta columna:

In [ ]:
def dife(s):
  return s.max() - s.min()

df.groupby('make').price.agg(["min","max", dife]).sort_values(ascending= False, by='dife')

,min,max,dife
make,,,
bmw,16430.0,41315.0,24885.0
mercedes-benz,25552.0,45400.0,19848.0
porsche,22018.0,37028.0,15010.0
nissan,5499.0,19699.0,14200.0
mazda,5195.0,18344.0,13149.0
toyota,5348.0,17669.0,12321.0
audi,13950.0,23875.0,9925.0
volvo,12940.0,22625.0,9685.0
mitsubishi,5389.0,14869.0,9480.0


Ahora, *groupby* nos permite agrupar por más de una clave. Agrupemos ahora por constructura y gasto de combustible en ciudad "city-mpg" *(miles per gallon)* para obtener los valores mínimos, máximos y su respectiva diferencia para los precios:

In [ ]:
df.groupby(['make', 'city-mpg']).price.agg(["min","max", dife])

min      max    dife
make        city-mpg                          
alfa-romero 19        16500.0  16500.0     0.0
            21        13495.0  16500.0  3005.0
audi        17        23875.0  23875.0     0.0
            18        17450.0  17450.0     0.0
            19        15250.0  18920.0  3670.0
...                       ...      ...     ...
volvo       18        21485.0  21485.0     0.0
            19        19045.0  22625.0  3580.0
            23        12940.0  16845.0  3905.0
            24        15985.0  16515.0   530.0
            26        22470.0  22470.0     0.0

[94 rows x 3 columns]

Note que en este caso hemos obtenido un DataFrame que contiene no un índice sino dos. Más adelante estudiaremos este tipo de estructuras.

**Ejercicio 1**

Construya un DataFrame a partir del siguiente diccionario 

`exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'], 'score': [12.5, 9, 16.5, 10, 9, 20, 14.5, 12, 8, 19], 'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1], 'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}` 

con las siguientes etiquetas para los índices 

`labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']`

* Seleccione solo las columnas *name* y *score*.
  
* Filtre las filas donde el número de intentos (attemps) sea mayor que 2.

* Filtre las filas donde el *score* se encuentre entre 15 y 20 (incluidos)

* Cambie el *score* en la fila 'd' a 11.5

* Agregue una nueva fila con etiqueta 'k' al dataframe con un valor que usted desee para cada columna

* Borre la fila 'k' y entregue nuevamente el DataFrame original.

* Ordene el DataFrame por nombre en orden ascendente.

* Reemplace los valores de la columna *qualify* que contiene los valores *yes* y *no* por los valores booleanos True y False respectivamente.

**Ejercicio 2:**

Importe el dataset `athletes` del repositorio del curso y realice las siguientes tareas:

- Importarlo como nombre athletes
- Filtre el dataframe tal que solo se quede con los atletas que hayan ganado al menos una medalla.
- Calcule la masa muscular por fila con una función como

$$I_{m.m.} = \frac{Weight}{Height^2}$$
- Agrupe los datos por calculando la media por deporte para cada una del resto de características. ¿Cuál es el deporte en el que los jugadores tienen mayor peso promedio?

- ¿Cuál es el pais con más participación femenina?